In [31]:
#%pip install mlflow boto3 awscli

In [32]:
import mlflow
# step 1 : Set up the mlflow tracking server 
mlflow.set_tracking_uri("http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/")

In [33]:
mlflow.set_experiment("Exp 2 - BOW vs TFIDF")

<Experiment: artifact_location='s3://youtube-recommender-exp-bucket/2', creation_time=1763813353748, experiment_id='2', last_update_time=1763813353748, lifecycle_stage='active', name='Exp 2 - BOW vs TFIDF', tags={}>

In [34]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os


In [35]:
df = pd.read_csv("C:/Users/mukti/Desktop/youtube_sentiment_analysis/youtube_preprocessing.csv").dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [36]:
#Step 0:train test split
X_train,X_test,y_train,y_test = train_test_split(
        df['clean_comment'],df['category'],
        test_size=0.2,
        random_state=42,
        stratify=df['category']
    )

# step 1 : function to run the experiment
def run_experiment(vectorizer_type,ngram_range,vectorizer_max_features,vectorizer_name,X_train_raw, X_test_raw,y_train_raw,y_test_raw):
    # step 2: Vectorization
    if vectorizer_type =="BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range,max_features=vectorizer_max_features)
    else:
        vectorizer =TfidfVectorizer(ngram_range=ngram_range,max_features=vectorizer_max_features)


    X_train_vec = vectorizer.fit_transform(X_train_raw)
    X_test_vec= vectorizer.transform(X_test_raw)

    # step 4: Define and train a Random forest model
    with mlflow.start_run() as run:
        #set tag for experiment and run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        #add a description
        mlflow.set_tag("description", f"Random forest with {vectorizer_name},ngram_range ={ngram_range},max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # RF params
        n_estimators = 150
        max_depth = 15
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Train model
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42
        )
        model.fit(X_train_vec, y_train_raw)

        #predict
        y_pred = model.predict(X_test_vec)

        #log accuracy
        accuracy =accuracy_score(y_test_raw,y_pred)
        mlflow.log_metric("accuracy",float(accuracy))

        #log classification report 
        classification_rep=classification_report(y_test_raw,y_pred,output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric_name, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric_name}", float(value))

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test_raw,y_pred)
        plt.figure(figsize=(8,6))
        sns.heatmap(conf_matrix,annot=True,fmt='d',cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion matrix : {vectorizer_name}_{ngram_range}")
        
        # Create folder if it doesn't exist
        os.makedirs("confusion_matrix", exist_ok=True)
        filename = f"confusion_matrix/conf_matrix_{vectorizer_type}_{ngram_range}.png"
        plt.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

# step 6: Run the experiment for BOW and TF-IDF with different n grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigrams, bigrams, trigrams
max_features = 5000  # Example max feature size

for ngram_range in ngram_ranges:
    #BOW Experiment
    run_experiment("BoW", ngram_range, max_features,"BoW",X_train,X_test,y_train,y_test)
    
    #TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features,"TF-IDF",X_train,X_test,y_train,y_test)



2025/11/22 18:23:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/2/runs/4b78024bf2984d22b81a3b5b8ff16ecb
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/2


2025/11/22 18:23:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/2/runs/2e80c856fb2042cdbf1013e83bd2b31d
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/2


2025/11/22 18:24:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/2/runs/7b7f95c32bed4432899f0d2f190624fc
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/2


2025/11/22 18:25:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/2/runs/c109ec53efb94d6fbbec2fa2bba6bf06
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/2


2025/11/22 18:25:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/2/runs/096269b5d7984870a30d43a91676dad3
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/2
